# Backtest sàn HNX

In [2]:
# Nhập thư viện
from vnstock import *
import pandas as pd
import pandas_ta as ta
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import warnings
from vnstock import stock_screening_insights

# Chặn warnings sharp ratio cho những cổ phiếu có returns = 0%
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Lọc cổ phiếu
list = []
params = {
    "exchangeName": "HNX",
    "avgTradingValue5Day": (10, 5000),
    "marketCap": (100, 500000),
}

data = stock_screening_insights(params, size=50)

# Set up và backtest hàng loạt
for i in data['ticker']:
    df = stock_historical_data(symbol=i, start_date="2024-03-01", end_date="2024-06-30", resolution="1D", type="stock", beautify=True, decor=False, source='DNSE')

    # Kiểm tra xem df có dữ liệu không
    if df is None or df.empty:
        print(f"Không có dữ liệu cho cổ phiếu {i}, bỏ qua.")
        continue

    df['time'] = pd.to_datetime(df['time'], format="%m/%d/%Y")
    df = df[df.high != df.low]
    df.set_index("time", inplace=True)

    # Tính toán EMA, Bollinger Bands, MACD và RSI
    df["EMA_12"] = ta.ema(df.close, length=12)
    df["EMA_26"] = ta.ema(df.close, length=26)
    df["MACD"], df["MACD_signal"], df["MACD_diff"] = ta.macd(df.close, fast=12, slow=26, signal=9)
    df["RSI"] = ta.rsi(df.close, length=14)

    # Chuyển đổi cột MACD_diff về dạng float (số) nếu có dữ liệu kiểu chuỗi
    df['MACD_diff'] = pd.to_numeric(df['MACD_diff'], errors='coerce')

    # Áp dụng tín hiệu MACD
    my_bbands = ta.bbands(df.close, length=14, std=1.5)
    df = df.join(my_bbands)  

    # Thay thế giá trị NaN bằng 0 mà không dùng inplace=True
    df = df.fillna(0)

    # Tắt cảnh báo chained_assignment
    pd.options.mode.chained_assignment = None

    # Hàm xác định tín hiệu EMA
    def ema_signal(df, current_candle, backcandles):
        df_slice = df.reset_index().copy()
        start = max(0, current_candle - backcandles)
        end = current_candle
        relevant_rows = df_slice.iloc[start:end]

        # Kiểm tra điều kiện để mua/bán dựa trên EMA
        if all(relevant_rows["EMA_12"] < relevant_rows["EMA_26"]):
            return 1  # bán
        elif all(relevant_rows["EMA_12"] > relevant_rows["EMA_26"]):
            return 2  # mua
        else:
            return 0
    
    # Hàm xác định tín hiệu Bollinger Bands
    def bol_signal(df, current_candle):
        if df.close.iloc[current_candle] >= df['BBU_14_1.5'].iloc[current_candle]:
            return 1  # bán
        elif df.close.iloc[current_candle] <= df['BBL_14_1.5'].iloc[current_candle]:
            return 2  # mua
        else:
            return 0

    # Hàm xác định tín hiệu MACD
    def macd_signal(df, current_candle):
        if df["MACD_diff"].iloc[current_candle] > 0 and df["MACD_diff"].iloc[current_candle - 1] <= 0:
            return 2  # mua khi MACD cắt đường tín hiệu từ dưới lên
        elif df["MACD_diff"].iloc[current_candle] < 0 and df["MACD_diff"].iloc[current_candle - 1] >= 0:
            return 1  # bán khi MACD cắt đường tín hiệu từ trên xuống
        else:
            return 0

    # Hàm xác định tín hiệu RSI
    def rsi_signal(df, current_candle):
        if df["RSI"].iloc[current_candle] > 70:
            return 1  # bán khi RSI trên 70 (mua quá mức)
        elif df["RSI"].iloc[current_candle] < 30:
            return 2  # mua khi RSI dưới 30 (bán quá mức)
        else:
            return 0
    
    # Áp dụng tín hiệu vào DataFrame
    df.reset_index(inplace=True)
    df['EMASignal'] = df.apply(lambda row: ema_signal(df, row.name, 2), axis=1)
    df['Bol_Signal'] = df.apply(lambda row: bol_signal(df, row.name), axis=1)
    df['MACD_Signal'] = df.apply(lambda row: macd_signal(df, row.name), axis=1)
    df['RSI_Signal'] = df.apply(lambda row: rsi_signal(df, row.name), axis=1)

    # Định dạng lại cột cho phù hợp với Backtesting
    df.index = pd.to_datetime(df['time'], format='%Y/%m/%d')
    df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})

    # Định nghĩa chiến lược
    class MyStrat(Strategy):
        def init(self):
            pass

        def next(self):
            # Kiểm tra tín hiệu mua từ EMA, Bollinger Bands, MACD, hoặc RSI
            if (self.data.EMASignal == 2 or self.data.Bol_Signal == 2 or 
                self.data.MACD_Signal == 2 or self.data.RSI_Signal == 2):  # Mua
                if len(self.trades) == 0:
                    self.buy()
            
            # Kiểm tra tín hiệu bán từ EMA, Bollinger Bands, MACD, hoặc RSI
            if (self.data.EMASignal == 1 or self.data.Bol_Signal == 1 or 
                self.data.MACD_Signal == 1 or self.data.RSI_Signal == 1):  # Bán
                if self.position.is_long:
                    self.position.close()

    # Thực hiện Backtest
    bt = Backtest(df, MyStrat, cash=10000000)
    run = bt.run()

    # In kết quả
    print(i, f'Lợi nhuận: {round(run["Return [%]"], 2)}%')
    list.append(run['Return [%]'])

# Tổng kết lợi nhuận
print(f'Tổng lợi nhuận: {sum(list)}%')


BVS Lợi nhuận: 76.22%
CEO Lợi nhuận: -16.67%
CTP Lợi nhuận: 113.2%
DL1 Lợi nhuận: 10.11%
DTD Lợi nhuận: -2.83%
HUT Lợi nhuận: 25.39%
IDC Lợi nhuận: 10.33%
LAS Lợi nhuận: 13.67%
MBS Lợi nhuận: 20.52%
NTP Lợi nhuận: 21.89%
PVS Lợi nhuận: -15.48%
SHS Lợi nhuận: 50.53%
TIG Lợi nhuận: 32.02%
TNG Lợi nhuận: 17.39%
VC3 Lợi nhuận: -0.66%
VGS Lợi nhuận: 23.55%
VTZ Lợi nhuận: 42.99%
Tổng lợi nhuận: 422.17294000000015%
